In [ ]:
'''
this program gives a report.csv file containing suspected numbers list, incoming and ougoing calls (counts 
and durations), total count and duration for a given number, how many IMEIs for a given number, how many calls
where first cell id and last cell id are same. Also you can generate the entire portion of cdr for 
suspected numbers.
this codes takes various inputs like threshold calls count and duration and can generate various types of
report based on requirements
To execute
copy this code and cdr file into
C:\\User\\Administrator or the default workspace of your compiler //report files are also creted here only//

to read the cdr file enterfile name in place of cdr1.csv in 3rd row  (hf=pd.read_csv('CDR1.csv))

set header=n where n is the row no.starting from 0, where columns names are there in the file
. In this case it is 3. For BSNL, Airtel put header =0.
put (sep='|',) inread csv line  for BSNL, Airtel.'''



import pandas as pd   #importing the modules
import numpy as np
import matplotlib.pyplot as plt

hf=pd.read_csv('cdr2.csv',header=3,dtype=object) #enter the filenamein place of cdr2.csv 
df=hf[['CALLING_NO','DURATION','FIRST_CELL','LAST_CELL','CALL_TYPE','ESN/IMEI']].copy()
column=['CALLING_NO','DURATION','FIRST_CELL','LAST_CELL','CALL_TYPE','IMEI']
df.columns=column
df['CALLING_NO']=df['CALLING_NO'].astype(str)
df=df.where((df['CALLING_NO'].str.len()==10) & (df['CALLING_NO'].str.get(0).isin(['6','7','8','9'])) ) #10 digit mobile number starting with 6-9 
df.CALLING_NO=pd.to_numeric(df.CALLING_NO,errors='coerce')
df.DURATION=pd.to_numeric(df.DURATION,errors='coerce')
df.IMEI=pd.to_numeric(df.IMEI,errors='coerce')
df.FIRST_CELL=pd.to_numeric(df.FIRST_CELL,errors='coerce')
df.LAST_CELL=pd.to_numeric(df.LAST_CELL,errors='coerce')

df=df.replace({'CALL_TYPE':{'MOC':1 ,'MTC':2,'SMO':3,'SMT':4}}) # check what are the call types in your cdr. use head command,
df=df[((df['CALL_TYPE']==1)|(df['CALL_TYPE']==2))]
df=df.fillna(0)
df.index=range(df.CALLING_NO.count())
df=df.applymap(int)




df['COUNT']=df.groupby('CALLING_NO')['CALLING_NO'].transform('count')
df['COUNT_IN']=df.where(df.CALL_TYPE==2).groupby('CALLING_NO')['CALLING_NO'].transform('count').fillna(0)
df['DUR_IN']=df.where(df.CALL_TYPE==2).groupby('CALLING_NO')['DURATION'].transform('sum').fillna(0)
df['COUNT_OUT']=df.where(df.CALL_TYPE==1).groupby('CALLING_NO')['CALLING_NO'].transform('count').fillna(0)
df['DUR_OUT']=df.where(df.CALL_TYPE==1).groupby('CALLING_NO')['DURATION'].transform('sum').fillna(0)
df['DUR_SUM']=df.groupby('CALLING_NO')['DURATION'].transform('sum')
df['DUR_AVG']=df.groupby('CALLING_NO')['DURATION'].transform('mean')
df['SAME_CELL']=df.where(df.FIRST_CELL==df.LAST_CELL).groupby('CALLING_NO')['CALLING_NO'].transform('count').fillna(0)
df['multiple_IMEI_count']=df.groupby('CALLING_NO')['IMEI'].transform('nunique')


df=df.groupby('CALLING_NO',as_index=False).agg('max')

In [ ]:
df['CALLING_NO'].nunique()

In [ ]:
plt.hist(df['DUR_AVG'],bins=[0,30,120,300],rwidth=0.95)
plt.xlabel("Avg call duration (in s)")
plt.ylabel("Number Of Calls")
plt.title("Average call duration plot")

In [ ]:
plt.hist(df['multiple_IMEI_count'],bins=[5,7,10,20],rwidth=0.95)
plt.xlabel("number of IMEIs")
plt.ylabel("Number Of subscribers")
plt.title("Average no. of IMEI plot")

In [ ]:
'''
print('input the lower calls count threshold')
a=int(input())
print('input the upper calls count threshold')
b=int(input())
print('input the lower total duration(in s) of all calls  threshold')
c=int(input())
print('input the upper total duration(in s) of all calls  threshold')
d=int(input())
print('input the lower multiple IMEI count threshold')
e=int(input())
print('input the upper multiple IMEI count  threshold')
f=int(input())
print('do you want report only where outgoing calls count is greater than incoming calls count? \nif yes, enter 1   see report_filtered.csv\n if you want report without this condition also enter 2 \notherwise enter any number')
g=int(input())
print('do you want complete portion of excel file for the the suspected numbers? \n if yes, enter 1 see sus_no_cdr.csv\n otherwise enter any number')
h=int(input())
'''
a=10;b=500;c=100;d=50000;e=4;f=7;g=2;h=0 # setting the thresholds for call count, duration, IMEIs 
df1=df[((df['DUR_SUM']>c)&(df['DUR_SUM']<d)&(df['COUNT']>a)&(df['COUNT']<b)&(df['multiple_IMEI_count']>e)&(df['multiple_IMEI_count']<f))]

df2=df1[['CALLING_NO','COUNT','COUNT_IN','COUNT_OUT','DUR_SUM','DUR_AVG','DUR_IN','DUR_OUT','SAME_CELL','multiple_IMEI_count']].copy()
cols=['sus_no','COUNT','COUNT_IN','COUNT_OUT','DUR_SUM','DUR_AVG','DUR_IN','DUR_OUT','SAME_CELL','multiple_IMEI_count']
df2.columns=cols
df2['DUR_AVG']=df2['DUR_AVG'].round(1)
df2.index=range(df2.sus_no.count())
df2=df2.drop_duplicates()
df2.index=range(df2.sus_no.count())








if(g==1):
    df3=df2[df2['COUNT_OUT']>df2['COUNT_IN']]
    df3.index=range(df3.sus_no.count())
    df3.to_csv('report_filtered.csv',index=False)# report where outgoing call count is greater than incoming call count
elif(g==2):
    df3=df2[df2['COUNT_OUT']>df2['COUNT_IN']]
    df3.index=range(df3.sus_no.count())
    df3.to_csv('report_filtered.csv',index=False)
    df2.to_csv('report.csv',index=False)
    
else:
    df2.to_csv('report.csv',index=False)



if(h==1):
    hf.CALLING_NO=pd.to_numeric(hf.CALLING_NO,errors='coerce')
    df4=hf.loc[hf['CALLING_NO'].isin(df2.sus_no)]
    df4.to_csv('sus_no_cdr.csv',index=False)
    


